## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [23]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Qaanaaq,GL,77.4840,-69.3632,-13.50,60,30,7.11,scattered clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,85.96,88,43,5.99,scattered clouds
2,2,Albany,US,42.6001,-73.9662,25.27,67,78,1.99,broken clouds
3,3,Cape Town,ZA,-33.9258,18.4232,66.27,81,20,4.61,few clouds
4,4,Airai,TL,-8.9266,125.4092,60.71,89,89,2.01,overcast clouds


In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,85.96,88,43,5.99,scattered clouds
5,5,Hobyo,SO,5.3505,48.5268,76.33,70,66,17.25,broken clouds
10,10,Atuona,PF,-9.8000,-139.0333,77.88,78,9,18.14,light rain
12,12,Hithadhoo,MV,-0.6000,73.0833,82.02,72,21,13.31,few clouds
16,16,Port Blair,IN,11.6667,92.7500,78.21,81,11,10.36,few clouds
19,19,Faanui,PF,-16.4833,-151.7500,81.59,75,100,20.33,overcast clouds
20,20,Bandarbeyla,SO,9.4942,50.8122,75.24,74,75,14.32,broken clouds
25,25,Avarua,CK,-21.2078,-159.7750,82.45,83,65,14.97,broken clouds
26,26,Macas,EC,-2.3167,-78.1167,76.75,64,75,4.61,broken clouds
27,27,Gouyave,GD,12.1646,-61.7296,79.57,78,20,16.11,few clouds


In [26]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                172
City                   172
Country                171
Lat                    172
Lng                    172
Max Temp               172
Humidity               172
Cloudiness             172
Wind Speed             172
Weather Description    172
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how='any')
preferred_cities_df.count()

City_ID                171
City                   171
Country                171
Lat                    171
Lng                    171
Max Temp               171
Humidity               171
Cloudiness             171
Wind Speed             171
Weather Description    171
dtype: int64

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,85.96,scattered clouds,-0.7393,-90.3518,
5,Hobyo,SO,76.33,broken clouds,5.3505,48.5268,
10,Atuona,PF,77.88,light rain,-9.8000,-139.0333,
12,Hithadhoo,MV,82.02,few clouds,-0.6000,73.0833,
16,Port Blair,IN,78.21,few clouds,11.6667,92.7500,
19,Faanui,PF,81.59,overcast clouds,-16.4833,-151.7500,
20,Bandarbeyla,SO,75.24,broken clouds,9.4942,50.8122,
25,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,
26,Macas,EC,76.75,broken clouds,-2.3167,-78.1167,
27,Gouyave,GD,79.57,few clouds,12.1646,-61.7296,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    #hotels = requests.get(base_url, params=params).json()
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
    # Grab the first hotel from the results and store the name.
    #try:
     #   hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    #except (IndexError):
     #   print("Hotel not found... skipping.")

#print(json.dumps(hotels, indent=4, sort_keys=True))
    

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(how='any')

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,85.96,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
5,Hobyo,SO,76.33,broken clouds,5.3505,48.5268,Halane Hotel and Restaurent
10,Atuona,PF,77.88,light rain,-9.8000,-139.0333,Villa Enata
12,Hithadhoo,MV,82.02,few clouds,-0.6000,73.0833,Scoop Guest House
16,Port Blair,IN,78.21,few clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
...,...,...,...,...,...,...,...
660,Miranda,CO,77.13,moderate rain,3.2499,-76.2281,unidiversidad organica la utopia
661,Pozo Colorado,PY,79.45,clear sky,-23.4923,-58.7972,Hospedaje la Amistad
669,Port Moresby,PG,78.22,haze,-9.4431,147.1797,Lamana Hotel (Port Moresby)
677,Makakilo City,US,80.92,light intensity drizzle,21.3469,-158.0858,Marriott's Ko Olina Beach Club


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description and Max Temp</dt><dd>{Weather Description} and {Max Temp}°F</dd>
</dl>
"""



# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))